## Getting Started
Importing libraries

In [2]:
# No DS without pd
import pandas as pd

# Import Matplotlib library and assigns shorthand 'plt'
import matplotlib
import matplotlib.pyplot as plt
# Imports Seaborn library and assigns shorthand 'sns'
import seaborn as sns
# Imports Plotly library and assigns shorthand 'px'
import plotly.express as px
# Imports Numpy library and assigns shorthand 'np'
import numpy as np

######## Shit below doesn't work########
# SQLite to not have to use a whole SQL Workbench but just runs out of a class, easier to code with
import sqlite3
# SQLite doesn't like mySQL format so I need to parse it
import pymysql
# pandassql lets you execute commands like in mySQL and is able to read stuff into an mySQL DB and converts it directly into a pd df
from pandasql import sqldf

## 1. Business Understanding
### Objective
The goal of this project is to understand and forecast product demand of a mexican toy store across different stores based on the internal company dataset.  
The dataset includes sales transactions, store information, inventory records, product details, and a calendar that helps align daily sales with time-based patterns such as holidays or promotions.

### Business Context
Fluctuations in demand directly impact how well stock levels, staffing, and promotions are planned.  
Without proper forecasting, stores risk overstocking slow-moving items or running out of high-demand products, both of which reduce operational efficiency.

### Analytical Objective
By analyzing historical sales patterns together with contextual data (store type, product category, inventory availability, and calendar effects), this project aims to build a data-driven demand forecasting model.  
The insights will support decisions in inventory planning, resource allocation, and promotional timing.

### Success Criteria
- **Quantitative:** Low forecast error (e.g., RMSE / MAPE) and stable model performance across stores.  
- **Qualitative:** Forecast outputs provide clear, actionable insights that can be used for planning stock, workforce, and logistics activities.


## 2. Data Understanding
This chapter is the initiialization of the mySQL DB and converts it into an pd df
This chapter does the first analysation of the DB

### 2.1 Initial data report
Reading data (train, test store)

In [2]:
# Create DB with SQLite
# HOST: Localhost DB_Name: mexico_toy_store_wh_db

# Create a connection to a newly created local database
conn = sqlite3.connect("mexico_toy_store_wh_db.db")
cursor = conn.cursor()

# Read your SQL file
with open(r"../sql/mexico_toy_store_wh_db.sql", 'r', encoding='utf-8') as f:
    sql_script = f.read()

# Execute all SQL commands in the file
cursor.executescript(sql_script)
conn.commit()


OperationalError: near "=": syntax error

In [60]:
# I just read the CSV files in until we got the connection properly working
base_path = r'../data/'

calendar_df = pd.read_csv(base_path + 'calendar.csv')
# data_dictionary_df = pd.read_csv(base_path + 'data_dictionary.csv')  # Not needed useless reference guide
inventory_df = pd.read_csv(base_path + 'inventory.csv')
products_df = pd.read_csv(base_path + 'products.csv')
sales_df = pd.read_csv(base_path + 'sales.csv')
stores_df = pd.read_csv(base_path + 'stores.csv')

### 2.2 Describing data (train, test, store)
Train data
Test data
Store data

In [61]:
# Delete comment about whatever you want to have more insights, some are still missing the print commands

#print(calendar_df.shape)
print(calendar_df.head())
#calendar_df.tail()
calendar_df.info()
#print(calendar_df.describe())
print(calendar_df.describe(include=object))
#print(calendar_df.dtypes)
print('_________________________________________________________________________________')
sales_df.shape
print(sales_df.head())
#sales_df.tail()
sales_df.info()
sales_df.describe()
sales_df.describe(include=object)
sales_df.dtypes
print('_________________________________________________________________________________')
inventory_df.shape
print(inventory_df.head())
#inventory_df.tail()
inventory_df.info()
inventory_df.describe()
#inventory_df.describe(include=object) # Throws ERROR: ValueError: No objects to concatenate
inventory_df.dtypes
print('_________________________________________________________________________________')
products_df.shape
print(products_df.head())
#products_df.tail()
products_df.info()
products_df.describe()
products_df.describe(include=object)
products_df.dtypes
print('_________________________________________________________________________________')
stores_df.shape
print(stores_df.head())
#stores_df.tail()
stores_df.info()
stores_df.describe()
stores_df.describe(include=object)
stores_df.dtypes
    

       Date
0  1/1/2022
1  1/2/2022
2  1/3/2022
3  1/4/2022
4  1/5/2022
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    638 non-null    object
dtypes: object(1)
memory usage: 5.1+ KB
            Date
count        638
unique       638
top     1/1/2022
freq           1
_________________________________________________________________________________
   Sale_ID        Date  Store_ID  Product_ID  Units
0        1  2022-01-01        24           4      1
1        2  2022-01-01        28           1      1
2        3  2022-01-01         6           8      1
3        4  2022-01-01        48           7      1
4        5  2022-01-01        44          18      1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829262 entries, 0 to 829261
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   

Store_ID            int64
Store_Name         object
Store_City         object
Store_Location     object
Store_Open_Date    object
dtype: object

Data looks good! No NULL values as it seems, which already looks like most of the data can be used. The connection of the Database could look like something like this:



### 2.3 Verify data quality
In this section we take a look at the quality of the data. Cleaning up for NULL values and dublicates.
#### 2.3.1 Missing values / Completeness

In [62]:
# Making sure that there is really no null values, calendar is beeing skipped, because there isn't really anyhing useful in there
print(sales_df.isnull().sum())
print(products_df.isnull().sum())
print(stores_df.isnull().sum())
print(inventory_df.isnull().sum())

Sale_ID       0
Date          0
Store_ID      0
Product_ID    0
Units         0
dtype: int64
Product_ID          0
Product_Name        0
Product_Category    0
Product_Cost        0
Product_Price       0
dtype: int64
Store_ID           0
Store_Name         0
Store_City         0
Store_Location     0
Store_Open_Date    0
dtype: int64
Store_ID         0
Product_ID       0
Stock_On_Hand    0
dtype: int64


In [63]:
# Making sure that there isn't dublicated values
print(sales_df.duplicated().sum())
print(products_df.duplicated().sum())
print(stores_df.duplicated().sum())
print(inventory_df.duplicated().sum())

0
0
0
0


In [64]:
# Checking for stores which haven't made any sales
sales_with_open = sales_df.merge(stores_df[['Store_ID', 'Store_Open_Date']], on='Store_ID', how='left')

open_zero_sales = sales_with_open[
    (sales_with_open['Date'] >= sales_with_open['Store_Open_Date']) &
    (sales_with_open['Units'] == 0)
]

print(open_zero_sales.shape, open_zero_sales.head())
print('_________________________________________________________________________________')
# Proofing validity by checking the opposite case, which has to have results
validity_check = sales_with_open[
    (sales_with_open['Date'] >= sales_with_open['Store_Open_Date']) &
    (sales_with_open['Units'] != 0)
]

print(validity_check.shape)
print(validity_check.head())

(0, 6) Empty DataFrame
Columns: [Sale_ID, Date, Store_ID, Product_ID, Units, Store_Open_Date]
Index: []
_________________________________________________________________________________
(829262, 6)
   Sale_ID        Date  Store_ID  Product_ID  Units Store_Open_Date
0        1  2022-01-01        24           4      1      2010-07-31
1        2  2022-01-01        28           1      1      2011-04-01
2        3  2022-01-01         6           8      1      2003-12-13
3        4  2022-01-01        48           7      1      2016-03-23
4        5  2022-01-01        44          18      1      2014-12-27


In [65]:
# Checking for stores which haven't opened yet
closed_stores = sales_with_open[sales_with_open['Date'] < sales_with_open['Store_Open_Date']]
print(closed_stores.shape)

(0, 6)


In [66]:
# Checking for referal integretiy
missing_prod = sales_df[~sales_df['Product_ID'].isin(products_df['Product_ID'])]
missing_store = sales_df[~sales_df['Store_ID'].isin(stores_df['Store_ID'])]
print(f'Missing sales --> prod references: {missing_prod.shape} and missing sales --> store references: {missing_store.shape}')
missing_prod = inventory_df[~inventory_df['Product_ID'].isin(products_df['Product_ID'])]
missing_store = inventory_df[~inventory_df['Store_ID'].isin(stores_df['Store_ID'])]
print(f'Missing inventory --> prod references: {missing_prod.shape} and missing inventory --> store references: {missing_store.shape}')
print('_________________________________________________________________________________')
# Verification that the function works, by deleting the 'not' aka ~
missing_prod = sales_df[sales_df['Product_ID'].isin(products_df['Product_ID'])]
missing_store = sales_df[sales_df['Store_ID'].isin(stores_df['Store_ID'])]
print(f'Not missing sales --> prod references: {missing_prod.shape} and not missing sales --> store references: {missing_store.shape}')
missing_prod = inventory_df[inventory_df['Product_ID'].isin(products_df['Product_ID'])]
missing_store = inventory_df[inventory_df['Store_ID'].isin(stores_df['Store_ID'])]
print(f'Not missing inventory --> prod references: {missing_prod.shape} and not missing inventory --> store references: {missing_store.shape}')

Missing sales --> prod references: (0, 5) and missing sales --> store references: (0, 5)
Missing inventory --> prod references: (0, 3) and missing inventory --> store references: (0, 3)
_________________________________________________________________________________
Not missing sales --> prod references: (829262, 5) and not missing sales --> store references: (829262, 5)
Not missing inventory --> prod references: (1593, 3) and not missing inventory --> store references: (1593, 3)


In [67]:
# Checking for negative values which just don't make any sense in a sales dataset
print("Negative units:", (sales_df['Units'] < 0).sum())
print("Negative stock:", (inventory_df['Stock_On_Hand'] < 0).sum())
#print("Negative or zero price:", (products_df['Product_Price'] <= 0).sum()) # Encountering an ERROR, Product_Price is storred as Str

Negative units: 0
Negative stock: 0


In [68]:
# Converting Str to float
products_df['Product_Price'] = products_df['Product_Price'].str.replace('$', '').astype(float)
products_df['Product_Price'] = products_df['Product_Price'].astype(float)
#print(products_df['Product_Price']) # Replacing Str with float works well, when error, just load the import of the data again

products_df['Product_Cost'] = products_df['Product_Cost'].str.replace('$', '').astype(float)
products_df['Product_Cost'] = products_df['Product_Cost'].astype(float)
#print(products_df['Product_Cost']) 

0      9.99
1      9.99
2      1.99
3      9.99
4      7.99
5      6.99
6     11.99
7      3.99
8      9.99
9     10.99
10    10.99
11     8.99
12    14.99
13     5.99
14     3.99
15     2.99
16    13.99
17    34.99
18    13.99
19     8.99
20     6.99
21    13.99
22     4.99
23    14.99
24     1.99
25    20.99
26     3.99
27     3.99
28     8.99
29    17.99
30     7.99
31    11.99
32    10.99
33    20.99
34     3.99
Name: Product_Cost, dtype: float64


In [69]:
# Continue checking for values which just don't make any sense
print("Negative or zero price:", (products_df['Product_Price'] <= 0).sum())
print("Price < Cost:", (products_df['Product_Price'] < products_df['Product_Cost']).sum())

Negative or zero price: 0
Price < Cost: 0


The dataset looks already very clean when it comes to the obvious issues, the prices in the products dataframe where changed to float, that they are comparable. For clarification all floats are ment to be in USD.
We checked now for:
- Any NULL values and didn't find any
- Any dublicates and didn't find any
- Any shops which have 0 sales (No)
- Are there shops which aren't open yet (No)
- Do all foreign keys have a primary key (Yes)
- Is there any negative stock or sales
- Are there any prices which don't make sense (ne

#### 2.3.2 Outliers detection / Accuracy

## Data Prept Delete rows from train data for closed stores
2.3.3 Data Quality Report

## 2.4 First Data Exploration

## 2.5 Data Preparation and further Data Exploration
Data Prepl Table Store, Column CompetitionDistance...
Data Prepl Table Store, Column Promo2SinceWeek...
Joining train and store data

## 2.5.1 Distribution

## 2.4.2 Correlations

## 2.4.3 Time-Series Analysis

## 2.5 Data Exploration Report

## 3. Data Preparation

## Dataset Description

## 3.1 Select Data

## 3.2 Clean Data

## 3.3 Construct Data

## 3.4 Integrate Data

## 3.5 Format Data

## 4. Modelling

## 4.1 Select Modelling

## 4.2 Techniques

## 4.3 Generate Test Design

## 4.4 Build Model

## 4.5 Assess Model

## 5. Evaluation

## 5.1 Evaluate Results

## 5.2 Review Process

## 5.3 Determine Next Steps

## 6. Deployment / Finalization

## 6.1 Plan Deployment

## 6.2 Plan Monitoring and Maintenance

## 6.3 Produce Final Report

## 6.4 Review Project